The New York Bike Load dataset contains the bike load information of 4 different bridges, Brooklyn Bridge, Manhattan Bridge, Williamsburg Bridge and Queensboro Bridge with given information on date, day, high temperature, low temperature and precipitation.

In my frequent Itemset Analysis, I would like to examine the relationship between temperature, Precipitation and the bike load on Manhattan Bridge.  I would like to know in most cases what causes increment or decrement of bike load on the bridge. When temperature goes up, and the precipitation also goes up, does the bike load goes up or down in most of the cases? What will happen to bike load in most of time when the temperature and the precipitation both go up?

To examine these relationship, we need change the data in the way that represent the relationship between 2 days. If the temperature goes up, we write 1. If the temperature goes down, we write -1, if it’s not changing, we write 0. After that we can use PyMing package to determine which combination of -1, 0, 1 occur the most frequently. 


## Step1: Get the data needed from database 

In [1]:
import sqlite3
import pandas as pd
from pprint import pprint
from pandas import DataFrame
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
import numpy as np
conn = sqlite3.connect('bicycle.db')
c=conn.cursor()
c.execute('SELECT LoTemp, Precip, Manhattan FROM bicycle')
data=c.fetchall()

## Step2: Transform the Data
In this step, I need to create another dataframe, use for loop to fill in the datafram based on the old dataframe. 

In [2]:
Data=DataFrame(data, columns=[ 'LoTemp', 'Precip', 'Manhattan'])
n=len(Data)
newData = DataFrame( index=range(0,n-1),columns=[ 'LoTemp', 'Precip', 'Manhattan'])

Data.loc[1,'LoTemp']
newData.loc[1,'LoTemp']=1

In [12]:
for x in range(0,n-1):
    for y in [ 'LoTemp', 'Precip', 'Manhattan']:
        if (Data.loc[x,y]<Data.loc[x+1,y]):
            newData.loc[x, y]=1
        elif (Data.loc[x,y]>Data.loc[x+1,y]):
            newData.loc[x, y]=-1 
        else:
            newData.loc[x, y]=0
datalist=newData.values.tolist()
datalist

[[-1, 1, -1],
 [-1, -1, -1],
 [-1, 1, -1],
 [-1, -1, 1],
 [1, 0, 1],
 [1, 1, -1],
 [-1, -1, 1],
 [-1, 1, -1],
 [-1, -1, 1],
 [1, 1, 1],
 [-1, 1, -1],
 [-1, -1, 1],
 [1, 0, -1],
 [-1, 0, -1],
 [1, 0, -1],
 [1, 0, 1],
 [1, 0, 1],
 [1, 0, 1],
 [-1, 0, -1],
 [0, 0, -1],
 [1, 1, -1],
 [-1, -1, 1],
 [1, 0, 1],
 [-1, 1, -1],
 [0, -1, 1],
 [1, 0, -1],
 [-1, 1, 1],
 [1, -1, 1],
 [-1, 1, -1],
 [1, -1, 1],
 [1, 1, -1],
 [-1, -1, 1],
 [-1, -1, 1],
 [1, 1, -1],
 [1, -1, 1],
 [1, 1, -1],
 [1, -1, 1],
 [-1, 0, -1],
 [1, 0, 1],
 [1, 0, -1],
 [1, 1, -1],
 [-1, -1, 1],
 [1, 0, 1],
 [-1, 0, 1],
 [0, 1, -1],
 [-1, 1, 1],
 [1, -1, 1],
 [1, 1, -1],
 [1, -1, 1],
 [1, 0, -1],
 [1, 0, -1],
 [-1, 0, -1],
 [-1, 1, -1],
 [1, -1, 1],
 [-1, 0, 1],
 [-1, 0, -1],
 [1, 1, -1],
 [1, 1, 1],
 [-1, 1, -1],
 [1, -1, 1],
 [-1, 1, -1],
 [1, -1, 1],
 [1, 0, -1],
 [1, 0, -1],
 [1, 0, 1],
 [-1, 0, 1],
 [1, 0, 1],
 [1, 0, -1],
 [1, 1, -1],
 [0, -1, 1],
 [-1, 0, -1],
 [1, 0, -1],
 [-1, 0, 1],
 [1, 0, 1],
 [0, 0, -1],
 [1, 0, 1],


## Step3: PyMining 

In [6]:
from pymining import itemmining
transactions = (('a', 'b', 'c'), ('b'), ('a'), ('a', 'c', 'd'), ('b', 'c'), ('b', 'c'))
relim_input = itemmining.get_relim_input(transactions)
report = itemmining.relim(relim_input, min_support=2)
report

{frozenset({'L'}): 1,
 frozenset({'L', 'T'}): 1,
 frozenset({'L', 'm'}): 1,
 frozenset({'L', 'T', 'm'}): 1,
 frozenset({'L', 'T', 'o'}): 1,
 frozenset({'L', 'T', 'm', 'o'}): 1,
 frozenset({'L', 'T', 'e', 'o'}): 1,
 frozenset({'L', 'T', 'e', 'm', 'o'}): 1,
 frozenset({'L', 'T', 'm', 'o', 'p'}): 1,
 frozenset({'L', 'T', 'e', 'm'}): 1,
 frozenset({'L', 'T', 'm', 'p'}): 1,
 frozenset({'L', 'e', 'm'}): 1,
 frozenset({'L', 'p'}): 1,
 frozenset({'L', 'm', 'p'}): 1,
 frozenset({'L', 'T', 'e', 'm', 'p'}): 1,
 frozenset({'L', 'o', 'p'}): 1,
 frozenset({'L', 'T', 'o', 'p'}): 1,
 frozenset({'L', 'e'}): 1,
 frozenset({'L', 'T', 'e'}): 1,
 frozenset({'L', 'T', 'p'}): 1,
 frozenset({'L', 'T', 'e', 'p'}): 1,
 frozenset({'L', 'o'}): 1,
 frozenset({'L', 'T', 'e', 'o', 'p'}): 1,
 frozenset({'L', 'm', 'o'}): 1,
 frozenset({'L', 'e', 'm', 'o'}): 1,
 frozenset({'L', 'm', 'o', 'p'}): 1,
 frozenset({'L', 'e', 'p'}): 1,
 frozenset({'L', 'e', 'm', 'p'}): 1,
 frozenset({'L', 'e', 'o'}): 1,
 frozenset({'L', 'e', 

In [ ]:
conn.close() 